# Adaptive Intelligence COM3240

## Lab 8: Reinforcement Learning


### Introduction

The main characteristic of reinforcement learning is that learning comes from interaction. The learner (or agent) has to achieve a specific goal by performing correct actions. If the goal is reached then he obtains reward. Given that there is no prior knowledge of which action is correct or wrong the agent needs to explore in order to reach the reward. It is possible for the agent to be rewarded not only when he reached the goal (future reward) but also by performing correct actions towards the goal (immediate reward).


### Basic concepts and algorithms

Reinforcement learning uses functions called value functions. Value functions are state-action pair functions that estimate how good a particular action will be in a given state, under a policy. Mathematically they are described by the notation $Q^{\pi}(s,a)$.

The value functions can be estimated by temporal difference (TD) learning methods. The TD methods calculate an estimation of the final reward at each state and the state-action value is updated for every step. They can be split up into two broad categories of methods, the on-policy methods and the off-policy methods. In the first category the agent learns the value of the policy that is used to make decisions, while in the second category the agent can learn different policies for behaviour and estimation. An example of an off-policy algorithm for TD learning is the Q-learning algorithm while an example of an on-policy algorithm for TD learning is the SARSA algorithm. The procedural forms of these two algorithms are described below:

<img src="./algoQ.png" width="500"/>

<img src="./algoSARSA.png" width="500"/>

where $\eta$ is the learning rate, $\gamma$ is the discount factor and $max_a$ is the maximum reward that is attainable in the state following the current one.


### The $\epsilon$-greedy policy

SARSA, being an on-policy algorithm does not explore new actions but always chooses the action with the highest expected reward. A way to include exploration is to choose the action with the highest expected reward at first, and then have a probability of following the same action or another, random one, the rest of the time. This approach is achieved with the $\epsilon$-greedy policy where a greedy action is selected with probability $1-\epsilon$, where $0 \leq \epsilon \leq 1$, and a random action is selected the rest of the times.


### Eligibility trace and the SARSA($\lambda$) algorithm 

When TD learning algorithms like SARSA observes a high or low reward, the only state-action pair which has its $Q(s,a)$ value updated is the state-action pair $(s,a)$ which leads to the reward. The next time the algorithm reaches state $s$, the state-action pair which leads to state $s$ will have its Q-value updated. The idea now is to update all Q-values of the trajectory once the reward is received. In order for this to be achieved an eligibility trace $\epsilon(s,a)$ needs to be maintained for each state-action pair. This trace indicates how much the current reward will influence $Q(s,a)$. The eligibility trace can be used in combination with the SARSA algorithm. The procedural form of the new algorithm named SARSA($\lambda$) is described below:

<img src="./algoSARSAl.png" width="500"/>

where $\eta$ is the learning rate, $\gamma$ is the discount factor and $\lambda$ is an indicator for the influence of the current reward.

## Laboratory 8

#### Exercise

<img src="./robot.png" width="500"/>

A robot moves in a square room with four possible actions, North, South, East and West. It has to learn a ”homing” task, i.e. to return to a particular location, where for instance it can charge its battery (reward location). There are no explicit landmarks, but to simplify the task we assume that the robot has been familiarised with the environment and therefore it has some internal representation of its position own position in the space, but no explicit memory of the reward location. Your task is to write a program where the above-mentioned goal-oriented behaviour (homing) can be learned by using a reinforcement learning algorithm, in the following way:

1. The robot is placed at a random location (segment) of the room.
2. It explores the space/learns the goal oriented behaviour by using the SARSA algorithm with Q-values.
3. Reward is given when the robot reaches the segment where the charger is located.
3. Trial ends when reward is reached (or a predefined number of steps is exceeded) and procedure starts from 1 until a predefined number of trials is reached.

At the end, you should plot the number of steps it took the robot to reach its target vs the trial number (learning curve). Successful learning means that the required number of steps is reduced as the trial number increases. We will call this procedure one run of the algorithm.

After developing the main routine (and confirming that your robot learns the desirable behaviour), use your code to study the properties of your model:

1. Implement an eligibility trace. Plot the difference in performance.
2. Find the optimal learning rate, discount factor and epsilon for the $\epsilon$-Greedy algorithm.

#### Script that returns the learnign curve for each trial

In [1]:
import numpy as np

def homing_nn(n_trials,n_steps,learning_rate,eps,gamma):
    ## Definition of the environment
    N = 7                               #height of the gridworld ---> number of rows
    M = 10                              #length of the gridworld ---> number of columns
    N_states = N * M                    #total number of states
    states_matrix = np.eye(N_states)
    N_actions = 4                                           #number of possible actions in each state: 1->N 2->E 3->S 4->W
    action_row_change = np.array([-1,0,+1,0])               #number of cell shifted in vertical as a function of the action
    action_col_change = np.array([0,+1,0,-1])               #number of cell shifted in horizontal as a function of the action
    End = np.array([1, 1])                                  #terminal state--->reward
    s_end = np.ravel_multi_index(End,dims=(N,M),order='F')  #terminal state. Conversion in single index

    ## Rewards
    R = 10                              #only when the robot reaches the charger, sited in End state

    ## Variables
    weights = np.random.rand(N_actions,N_states)
    learning_curve = np.zeros((1,n_trials))

    ## SARSA

    # Start trials
    for trial in range(n_trials):

        # Initialization
        Start = np.array([np.random.randint(N),np.random.randint(M)])   #random start
        s_start = np.ravel_multi_index(Start,dims=(N,M),order='F')      #conversion in single index
        state = Start                                                   #set current state
        s_index = s_start                                               #conversion in single index
        step = 0

        # Start steps
        while s_index != s_end and step <= n_steps:

            step += 1
            learning_curve[0,trial] = step

            input_vector = states_matrix[:,s_index].reshape(N_states,1)         #convert the state into an input vector

            #compute Qvalues. Qvalue=logsig(weights*input). Qvalue is 2x1, one value for each output neuron
            Q = 1 / ( 1 + np.exp( - weights.dot(input_vector)))    #Qvalue is 2x1 implementation of logsig

            #eps-greedy policy implementation
            greedy = (np.random.rand() > eps)               #1--->greedy action 0--->non-greedy action
            if greedy:
                action = np.argmax(Q)                           #pick best action
            else:
                action = np.random.randint(N_actions)           #pick random action


            state_new = np.array([0,0])
            #move into a new state
            state_new[0] = state[0] + action_row_change[action]
            state_new[1] = state[1] + action_col_change[action]

            #put the robot back in grid if it goes out. Consider also the option to give a negative reward
            if state_new[0] < 0:
                state_new[0] = 0
            if state_new[0] >= N:
                state_new[0] = N-1
            if state_new[1] < 0:
                state_new[1] = 0
            if state_new[1] >= M:
                state_new[1] = M-1

            s_index_new = np.ravel_multi_index(state_new,dims=(N,M),order='F')  #conversion in a single index

            ## TODO update Qvalues. Only if is not the first step

            #store variables for sarsa computation in the next step
            output = np.zeros((N_actions,1))
            output[action] = 1

            #update variables
            input_old = input_vector
            output_old = output
            Q_old = Q[action]
            r_old = 0

            state[0] = state_new[0]
            state[1] = state_new[1]
            s_index = s_index_new

            ## TODO: check if state is terminal and update the weights consequently
            if s_index == s_end:
                pass


    return learning_curve


#### Call the function homing_nn


In [2]:
#NBVAL_SKIP

# Parameter setup
nrepetitions = 15;  # number of runs for the algorithm
nTrials = 1000;     # should be integer >0
nSteps = 50;        # maximum number of allowed steps
learningRate = 0.3; # should be real, Greater than 0
epsilon = 0.25;     # should be real, Greater or Equal to 0; epsion=0 Greedy, otherwise epsilon-Greedy
gamma = 0.9;        # should be real, positive, smaller than 1


#TODO: average number of steps to finish the task per trial for nrepetitions
#for i = 1:nrepetitions....
homing_nn(nTrials,nSteps,learningRate,epsilon,gamma)




array([[51., 51., 11., 51., 51., 51.,  2., 51., 51., 51., 51., 51., 51.,
        51., 51., 51., 51., 21., 51., 51., 51., 39., 51., 51., 13., 51.,
        51., 51., 51., 51., 51., 51., 51., 51., 51., 51., 51., 19., 51.,
        51., 51., 44., 51., 51., 51., 51., 51.,  7., 51., 51., 51., 51.,
        15., 51., 51.,  1., 51., 51., 51., 51., 51., 51., 51., 51., 18.,
        26., 51., 26., 51., 51., 51., 51., 51., 51., 51., 51., 51., 51.,
        51., 51., 51., 51., 51., 51., 51., 51., 51., 51., 51., 51., 51.,
        51., 51., 51., 51., 11., 51.,  3., 51., 51., 51., 51.,  8., 49.,
        51., 51., 51., 51., 51., 51.,  1., 51., 51.,  4., 51.,  4., 51.,
        51.,  3., 17.,  6., 51., 51., 51., 51., 51.,  3., 51., 51., 51.,
        51., 51., 51., 51., 51., 25., 51., 51., 48., 51., 51., 20., 51.,
        51.,  4., 51., 51., 51.,  2., 10., 51.,  4., 51., 51., 41., 13.,
        51., 51., 51.,  3., 51., 51., 29., 51., 51.,  1., 51., 51., 51.,
        51., 51., 51.,  2., 51., 28., 30., 51., 51.